In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.model import fit
from fastai.dataset import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle
import random

# Fit language model on Wordpress

In [4]:
bs, bptt = 64, 70

In [3]:
#python -m spacy download fr
my_tok = spacy.load('fr')

def my_spacy_tok(x):
    return [tok.text for tok in my_tok.tokenizer(x)]

In [4]:
PATH = 'data/txt/wordpress'
TEXT = data.Field(lower=True, tokenize=my_spacy_tok)
md = LanguageModelData.from_text_files(PATH, TEXT, train='.', validation='.', test='.',
                                       bs=bs, bptt=bptt, min_freq=10)

In [5]:
len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(123, 4002, 1, 558973)

In [6]:
em_sz = 200
nh = 500
nl = 3
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))

In [7]:
learner = md.get_model(opt_fn, em_sz, nh, nl,
    dropout=0.05, dropouth=0.1, dropouti=0.05, dropoute=0.02, wdrop=0.2)
# dropout=0.4, dropouth=0.3, dropouti=0.65, dropoute=0.1, wdrop=0.5
#                dropouti=0.05, dropout=0.05, wdrop=0.1, dropoute=0.02, dropouth=0.05)
learner.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
learner.clip=0.3

In [8]:
warnings.filterwarnings('ignore',category=UserWarning)

In [9]:
learner.fit(3e-3, 1, wds=1e-6)
learner.save_encoder('wordpress_1')

epoch      trn_loss   val_loss   
    0      5.41836    5.1515    



In [10]:
learner.fit(3e-3, 4, wds=1e-6, cycle_len=1, cycle_mult=2)
learner.save_encoder('wordpress_2')

epoch      trn_loss   val_loss   
    0      4.964747   4.861962  
    1      4.710859   4.511409  
    2      4.499228   4.398448  
    3      4.389906   4.205618  
    4      4.214793   4.045292  
    5      4.100974   3.948904  
    6      4.032428   3.925189  
    7      4.065681   3.863705  
    8      3.962731   3.748533  
    9      3.858471   3.633356  
    10     3.764991   3.536353  
    11     3.685439   3.447273  
    12     3.616239   3.393011  
    13     3.578113   3.365949  
    14     3.576423   3.36475   



In [11]:
learner.fit(3e-3, 1, wds=1e-6, cycle_len=10)
learner.save_encoder('wordpress_3')

epoch      trn_loss   val_loss   
    0      3.621457   3.424829  
    1      3.614206   3.343866  
    2      3.550171   3.273521  
    3      3.47219    3.167678  
    4      3.415134   3.09017   
    5      3.342923   3.012518  
    6      3.283233   2.959568  
    7      3.243617   2.922631  
    8      3.208909   2.923379  
    9      3.20387    2.886638  



In [12]:
learner.fit(1e-3, 4, wds=1e-6, cycle_len=1, cycle_mult=2)
learner.save_encoder('wordpress_4')

epoch      trn_loss   val_loss   
    0      3.254793   2.883765  
    1      3.270128   2.881448  
    2      3.239029   2.847525  
    3      3.232687   2.858032  
    4      3.206249   2.821148  
    5      3.171746   2.772858  
    6      3.147078   2.755853  
    7      3.194739   2.779003  
    8      3.170617   2.7595    
    9      3.157458   2.715929  
    10     3.121693   2.688937  
    11     3.096725   2.668138  
    12     3.077755   2.633148  
    13     3.077113   2.623319  
    14     3.077111   2.6103    



In [13]:
learner.fit(1e-3, 1, wds=1e-6, cycle_len=10)
learner.save_encoder('wordpress_5')

epoch      trn_loss   val_loss   
    0      3.110921   2.705874  
    1      3.108543   2.645146  
    2      3.063471   2.617466  
    3      3.111695   2.619941  
    4      3.034176   2.557742  
    5      3.015316   2.519851  
    6      2.985358   2.496241  
    7      2.965874   2.478436  
    8      2.943421   2.474756  
    9      2.976183   2.489776  



# Show most common predictions

In [14]:
learner.load_encoder('wordpress_5')
m = learner.model.cpu()

In [15]:
def proc_str(s): return TEXT.preprocess(TEXT.tokenize(s))
def num_str(s): return TEXT.numericalize([proc_str(s)])

In [16]:
s = "j'aime"
t = num_str(s)

In [17]:
# Set batch size to 1
m[0].bs=1
# Turn off dropout
m.eval()
# Reset hidden state
m.reset()
# Get predictions from model
res,*_ = m(t)
# Put the batch size back to what it was
m[0].bs=bs

In [18]:
nexts = torch.topk(res[-1], 10)[1]
[TEXT.vocab.itos[o] for o in to_np(nexts)]

['le', 'la', 'les', 'l’', 'se', 'vous', 'voyager', 'faire', 'pouvoir', 'leur']

In [19]:
nexts = torch.multinomial(res[-1].exp(), 10)
[TEXT.vocab.itos[o] for o in to_np(nexts)]

['goroka',
 'dire',
 'lire',
 'les',
 'suivre',
 'jouer',
 'planète',
 'tomber',
 'la',
 'améliorer']

# Sample the model

In [116]:
def sample_model(m, s, l=100, style='topk'):
    
    result = ''
    
    t = num_str(s)
    m[0].bs=1
    m.eval()
    m.reset()
    res,*_ = m(t)
    result += s + ' '

    for i in range(l):
        if style == 'topk':
            n = res[-1].topk(2)[1]
        else:
            n = torch.multinomial(res[-1].exp(), 2)
        n = n[1] if n.data[0]==0 else n[0]
        word = TEXT.vocab.itos[n.data[0]]
        result += word + ' '
        if word=='<eos>': break
        res,*_ = m(n[0].unsqueeze(0).unsqueeze(0))

    m[0].bs=bs
    
    return result

In [109]:
learner.load_encoder('wordpress_5')
m = learner.model.cpu()

In [117]:
sample_model(m, "je suis")

'je suis obligé de me faire plaisir . je ne sais pas si ce voyage est une ville très touristique , très grande , c’ est aussi la grande ville qui m’ a fait pour se faire inviter . je suis bien content de me voir seul et de me faire inviter à dormir chez l’ habitant . je me suis aussi fait un peu de temps pour moi , et pour un autre voyage en australie . je n’ ai pas eu de problème à me faire plaisir . je ne sais pas si ce voyage est une expérience très touristique '

In [119]:
sample_model(m, "je suis", style='multinomial')

'je suis telle avoir de l’ argent pour voyager , ces amis , plusieurs vins , quelques amis , equateur et articles , ou parlera de l’ ordinateur . tous me vous en donne   3 jours et rencontrés à mon blog , sans   pas besoin d’ un an et demi ,   car pourtant on s’ était bien déjà fait des frontières modernes . . sur le rêve , à moment du travail on part très bien , ils aiment \xa0 leur planète , la qualité principale nouvelle australien viennent a uluru et sydney coober avril . stella , un '

# Finetune on blogger data

In [24]:
PATH = 'data/txt/blogger'
#TEXT = data.Field(lower=True, tokenize=my_spacy_tok)
md2 = LanguageModelData.from_text_files(PATH, TEXT, train='.', validation='.', test='.',
                                       bs=bs, bptt=bptt, min_freq=10)

In [25]:
len(md2.trn_dl), md2.nt, len(md2.trn_ds), len(md2.trn_ds[0].text)

(20, 4002, 1, 95914)

In [82]:
em_sz = 200
nh = 500
nl = 3
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))

In [83]:
learner2 = md2.get_model(opt_fn, em_sz, nh, nl,
    dropout=0.05, dropouth=0.1, dropouti=0.05, dropoute=0.02, wdrop=0.2)
# dropout=0.4, dropouth=0.3, dropouti=0.65, dropoute=0.1, wdrop=0.5
#                dropouti=0.05, dropout=0.05, wdrop=0.1, dropoute=0.02, dropouth=0.05)
learner2.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
learner2.clip=0.3

In [84]:
learner2.load_encoder('wordpress_5')

In [29]:
learner2.fit(3e-3, 1, wds=1e-6)
learner2.save_encoder('blogger_1')

epoch      trn_loss   val_loss   
    0      4.07943    3.612095  



In [30]:
learner2.fit(3e-3, 4, wds=1e-6, cycle_len=1, cycle_mult=2)
learner2.save_encoder('blogger_2')

epoch      trn_loss   val_loss   
    0      3.729076   3.4321    
    1      3.649061   3.236375  
    2      3.53968    3.166946  
    3      3.483248   3.033893  
    4      3.402295   2.859933  
    5      3.293501   2.742957  
    6      3.212594   2.733846  
    7      3.17222    2.634837  
    8      3.112819   2.48527   
    9      3.032762   2.337774  
    10     2.938747   2.240985  
    11     2.830832   2.172124  
    12     2.736191   2.025528  
    13     2.659064   1.991474  
    14     2.600195   1.999561  



In [31]:
learner2.fit(3e-3, 1, wds=1e-6, cycle_len=10)
learner2.save_encoder('blogger_3')

epoch      trn_loss   val_loss   
    0      2.508398   2.028785  
    1      2.506911   1.910381  
    2      2.477282   1.858803  
    3      2.442942   1.696492  
    4      2.39218    1.615291  
    5      2.311126   1.52568   
    6      2.239762   1.448574  
    7      2.192392   1.445718  
    8      2.15074    1.409442  
    9      2.128195   1.430044  



In [32]:
learner2.fit(1e-3, 4, wds=1e-6, cycle_len=1, cycle_mult=2)
learner2.save_encoder('blogger_4')

epoch      trn_loss   val_loss   
    0      2.045861   1.391264  
    1      2.07245    1.361346  
    2      2.109661   1.374343  
    3      2.065975   1.356039  
    4      2.026473   1.310358  
    5      2.025779   1.267039  
    6      1.988473   1.310442  
    7      2.001189   1.421678  
    8      1.987376   1.272208  
    9      1.94994    1.184513  
    10     1.92364    1.159139  
    11     1.913576   1.145634  
    12     1.931912   1.139126  
    13     1.928751   1.177401  
    14     1.901622   1.257719  



In [33]:
learner2.fit(1e-3, 1, wds=1e-6, cycle_len=10)
learner2.save_encoder('blogger_5')

epoch      trn_loss   val_loss   
    0      1.806324   1.147617  
    1      1.825615   1.086717  
    2      1.792029   1.049193  
    3      1.825028   1.042164  
    4      1.783125   1.027698  
    5      1.786378   0.990423  
    6      1.781774   1.047837  
    7      1.764112   1.0085    
    8      1.72583    1.004652  
    9      1.732903   0.980324  



In [57]:
learner2.fit(3e-3, 6, wds=1e-6, cycle_len=1, cycle_mult=2)
learner2.save_encoder('blogger_6')

epoch      trn_loss   val_loss   
    0      1.847859   1.060709  
    1      1.813359   1.026883  
    2      1.778089   0.948474  
    3      1.824945   1.080708  
    4      1.800656   0.993749  
    5      1.737481   0.902956  
    6      1.709092   0.892642  
    7      1.695567   0.864817  
    8      1.700057   0.84268   
    9      1.672589   0.770739  
    10     1.623281   0.716308  
    11     1.584262   0.687014  
    12     1.523147   0.624581  
    13     1.483324   0.658361  
    14     1.474085   0.621477  
    15     1.520616   0.714385  
    16     1.520002   0.642848  
    17     1.515244   0.60481   
    18     1.472622   0.54816   
    19     1.449795   0.688151  
    20     1.420494   0.504746  
    21     1.417902   0.512269  
    22     1.363959   0.54308   
    23     1.324987   0.447386  
    24     1.280399   0.387428  
    25     1.230288   0.366208  
    26     1.175008   0.361138  
    27     1.143298   0.340434  
    28     1.104044   0.444046  
    29   

# Sample predictions

In [120]:
learner2.load_encoder('blogger_6')

In [121]:
m2 = learner2.model.cpu()

In [124]:
sample_model(m2, "je suis")

'je suis fatigué du stop qui ne marche pas , et je dois marcher pour la nuit . de toute façon , je vais en profiter pour faire un petit tour de terrain de foot très haut , mais vu que le paysage est en train de dormir dans le désert . je sais bien que le terrain est à peine pour sortir de la paz , mais je suis dans le vent . la douleur se termine plutôt . je suis un peu trop faible pour que ça soit normal , et je me fais embarquer en chemin par un groupe '

In [129]:
with open('data/sampled.txt', 'w') as f:
    f.write(sample_model(m2, "je suis", 100000))

# Fit language model on Blogger

In [61]:
bs, bptt = 64, 70

In [62]:
#python -m spacy download fr
my_tok = spacy.load('fr')

def my_spacy_tok(x):
    return [tok.text for tok in my_tok.tokenizer(x)]

In [63]:
PATH = 'data/txt/blogger'
TEXT = data.Field(lower=True, tokenize=my_spacy_tok)
md3 = LanguageModelData.from_text_files(PATH, TEXT, train='.', validation='.', test='.',
                                       bs=bs, bptt=bptt, min_freq=5)

In [64]:
len(md3.trn_dl), md3.nt, len(md3.trn_ds), len(md3.trn_ds[0].text)

(20, 1764, 1, 95914)

In [65]:
em_sz = 200
nh = 500
nl = 3
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))

In [66]:
learner3 = md3.get_model(opt_fn, em_sz, nh, nl,
    dropout=0.05, dropouth=0.1, dropouti=0.05, dropoute=0.02, wdrop=0.2)
# dropout=0.4, dropouth=0.3, dropouti=0.65, dropoute=0.1, wdrop=0.5
#                dropouti=0.05, dropout=0.05, wdrop=0.1, dropoute=0.02, dropouth=0.05)
learner3.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
learner3.clip=0.3

In [67]:
warnings.filterwarnings('ignore',category=UserWarning)

In [37]:
learner3.fit(3e-3, 1, wds=1e-6)
learner3.save_encoder('blogger2_1')

epoch      trn_loss   val_loss   
    0      5.716376   5.336585  


In [38]:
learner3.fit(3e-3, 4, wds=1e-6, cycle_len=1, cycle_mult=2)
learner3.save_encoder('blogger2_2')

epoch      trn_loss   val_loss   
    0      5.324759   5.272443  
    1      5.258127   5.057779  
    2      5.13788    4.964386  
    3      5.045314   4.746413  
    4      4.911257   4.603815  
    5      4.788528   4.515129  
    6      4.701424   4.500392  
    7      4.651704   4.432693  
    8      4.576051   4.340849  
    9      4.49755    4.248639  
    10     4.417539   4.172508  
    11     4.343906   4.123144  
    12     4.278645   4.081852  
    13     4.227803   4.066607  
    14     4.18771    4.062248  


In [39]:
learner3.fit(3e-3, 1, wds=1e-6, cycle_len=10)
learner3.save_encoder('blogger2_3')

epoch      trn_loss   val_loss   
    0      4.135802   4.05787   
    1      4.116346   3.977947  
    2      4.078284   3.91676   
    3      4.020767   3.828484  
    4      3.968137   3.752808  
    5      3.914007   3.696593  
    6      3.86487    3.654347  
    7      3.81588    3.624824  
    8      3.775475   3.604919  
    9      3.751243   3.605023  


In [40]:
learner3.fit(1e-3, 4, wds=1e-6, cycle_len=1, cycle_mult=2)
learner3.save_encoder('blogger2_4')

epoch      trn_loss   val_loss   
    0      3.725288   3.603257  
    1      3.70469    3.573659  
    2      3.693178   3.551518  
    3      3.686415   3.540763  
    4      3.668134   3.502769  
    5      3.651766   3.470252  
    6      3.62752    3.464709  
    7      3.61584    3.443684  
    8      3.599719   3.424744  
    9      3.583802   3.40861   
    10     3.55352    3.331029  
    11     3.536273   3.31895   
    12     3.504065   3.298577  
    13     3.488571   3.282547  
    14     3.471088   3.29126   


In [41]:
learner3.fit(1e-3, 1, wds=1e-6, cycle_len=10)
learner3.save_encoder('blogger2_5')

epoch      trn_loss   val_loss   
    0      3.439      3.298283  
    1      3.45457    3.250465  
    2      3.433826   3.227056  
    3      3.409497   3.18042   
    4      3.38546    3.142725  
    5      3.356613   3.120348  
    6      3.336234   3.090318  
    7      3.339717   3.082014  
    8      3.327456   3.075781  
    9      3.30387    3.074916  


In [42]:
learner3.fit(3e-3, 6, wds=1e-6, cycle_len=1, cycle_mult=2)
learner3.save_encoder('blogger2_6')

epoch      trn_loss   val_loss   
    0      3.401124   3.115943  
    1      3.394359   3.104966  
    2      3.320925   2.998596  
    3      3.324621   3.014998  
    4      3.274925   2.889136  
    5      3.203866   2.791183  
    6      3.143389   2.743696  
    7      3.133088   2.808684  
    8      3.115346   2.750989  
    9      3.078342   2.630241  
    10     3.007581   2.504309  
    11     2.938244   2.397932  
    12     2.85738    2.403038  
    13     2.787358   2.290239  
    14     2.738232   2.267984  
    15     2.755664   2.416386  
    16     2.761862   2.296825  
    17     2.726909   2.193772  
    18     2.686065   2.133616  
    19     2.651273   2.069843  
    20     2.582889   1.935177  
    21     2.515206   1.91445   
    22     2.453686   1.846575  
    23     2.371907   1.750518  
    24     2.321926   1.648522  
    25     2.262994   1.607323  
    26     2.207913   1.589909  
    27     2.163113   1.55609   
    28     2.124327   1.506065  
    29   

In [68]:
learner3.fit(3e-3, 4, wds=1e-6, cycle_len=1, cycle_mult=2)
learner3.load_encoder('blogger2_7')

In [69]:
learner3.fit(3e-3, 6, wds=1e-6, cycle_len=1, cycle_mult=2)
learner3.save_encoder('blogger2_8')

epoch      trn_loss   val_loss   
    0      1.170379   0.479152  
    1      1.292388   0.474547  
    2      1.261556   0.434445  
    3      1.257896   0.46974   
    4      1.261782   0.421744  
    5      1.233715   0.369263  
    6      1.162712   0.376096  
    7      1.163608   0.448127  
    8      1.192446   0.397199  
    9      1.178609   0.377584  
    10     1.134461   0.370582  
    11     1.085404   0.308952  
    12     1.031627   0.279289  
    13     1.005367   0.273756  
    14     1.029904   0.339459  
    15     1.03545    0.349991  
    16     1.032645   0.364726  
    17     1.100678   0.353906  
    18     1.053453   0.278231  
    19     1.073659   0.312367  
    20     1.033287   0.33833   
    21     0.99316    0.224752  
    22     0.94959    0.20221   
    23     0.945983   0.21263   
    24     0.935234   0.196404  
    25     0.885704   0.168797  
    26     0.87833    0.181851  
    27     0.876303   0.152599  
    28     0.818133   0.156572  
    29   

# Sample predictions

In [70]:
def proc_str(s): return TEXT.preprocess(TEXT.tokenize(s))
def num_str(s): return TEXT.numericalize([proc_str(s)])

In [71]:
def sample_model(m, s, l=100, style='topk'):
    
    result = ''
    
    t = num_str(s)
    m[0].bs=1
    m.eval()
    m.reset()
    res,*_ = m(t)
    result += s + ' '

    for i in range(l):
        if style == 'topk':
            n = res[-1].topk(2)[1]
        else:
            n = torch.multinomial(res[-1].exp(), 2)
        n = n[1] if n.data[0]==0 else n[0]
        word = TEXT.vocab.itos[n.data[0]]
        result += word + ' '
        if word=='<eos>': break
        res,*_ = m(n[0].unsqueeze(0).unsqueeze(0))

    m[0].bs=bs
    
    return result

In [72]:
learner3.load_encoder('blogger2_8')

In [73]:
m3 = learner3.model.cpu()

In [75]:
print(sample_model(m3, "je suis", l=300))

je suis fatigué et me dit que c' est 30 bolivianos ( en fait ) . la route ( ma batterie ) , j' arrive à la nuit . " je ne sais pas trop comment ça va . " je vais être de tente . - non . tu n' y pas pas . " je vois une petite boucle de 10 jours en stop . j' ai aussi une tente . cette nuit , j' ai dormi dans un village dans la montagne : " non , tu ne peux pas faire de stop ici . " ça me semble totalement bien . le lendemain , après avoir fait mes faire a la carte , je me fais la réflexion qu' il y a quand même des coins sympas pour camper . et puis , de fil en bus , je me suis fait embarquer par un conducteur qui s' appelle the me la part , et je fais du stop dans la route et j' aimerais aller en direction de mendoza . cette fois - ci , la remorque est pleine à plus , et avec nori on fait entre 2 km . cependant , j' ai une flemme de retour . je vais à l' une d' elle au hasard et trouve une femme . " bonjour , je cherche le camping . - oui , il est là . - d' accord , où se trouve la cart